# 15. 폼으로 포스트 작성과 수정 기능 구현
___
## 포스트 작성 페이지만들기
포스트 작성 페이지에는 방문자가 제목,내용 등을 입력할 수 있어야하며, 입력한 내용을 서버에 전송해 DB로 저장할 수 있는 빈 칸이 있어야한다. 
이러한 빈 칸을 장고에서는 '폼(form)'이라고 부른다.
### 포스트 작성 페이지 기본 틀 만들기
#### (1) 테스트 코드 작성
우선 테스트 코드 부터 작성한다. test_create_post() 라고 명명하고 /blog/create_post/ 라는 URL로 방문하면 작성 페이지가 정상적으로 열려야 한다.  
웹 브라우저의 타이틀은 'Creat Post - Blog' 라고 하고 마지막 메인 영역은 'Create New Post'라는 문구가 있도록 한다.

In [2]:
def test_create_post(self):
    response=self.client.get('/blog/create_post/')
    self.assertEqual(response.status_code, 200)
    soup=BeautifulSoup(response.content, 'html.parser')

    self.assertEqual('Create Post - Blog', soup.title.text)
    main_area=soup.find('div', id='main-area')
    self.assertIn('Create New Post',main_area.text)

이제 blog 앱의 views.py와 urls.py에 내용을 추가하여야 하며 장고에서 제공하는 기능을 사용해 CBV로 만들자.
#### (2) views.py에 CreateView 추가하기
먼저 blog/views.py를 열어 장고가 제공하는 CreateView를 상속받아 PostCreate라는 클래스를 만든다. model=Post로 Post모델을 사용하고, Post 모델에 사용할 필드명은 리스트로 작성해 fields 변수에 저장한다.

In [4]:
class PostCreate(CreateView):
    model=Post
    fields=['title', 'hook_text', 'content', 'head_image', 'file_upload', 'category']

NameError: name 'CreateView' is not defined

fields 리스트에는 author, created_at, tags는 넣지 않는다. 작성자는 당연히 본인이며 작성시간은 자동으로 채워지게 한다. tags필드는 나중에 방문자가 직접 텍스트로 입력하도록 추후에 구현한다.
#### (3) URL에 create_post/ cnrkgkrl
이제 blog/urls.py를 열고 방문자가 blog/creage_post/로 URL을 입력하는 경우 방금 만든 PostCreate 클래스를 사용하도록 수정한다.

In [5]:
path('create_post/',views.PostCreate.as_view())

NameError: name 'path' is not defined

#### (4) 템플릿 만들기
테스트를 하면 템플릿 파일이 없다고 나온다. post_form.html을 만들어 주어야 한다. base.html을 확장해서 만들자. 제일 위에 {% extends 'blog/base.html' %}을 입력하고 시작하면된다. 타이틀로 'Create Post - Blog'를 넣어주고 메인 영역은 form 태그를 넣어 작성한다. 그리고 {{form}}을 table 태그로 감싸준다.(CreateView를 상속받은 PostCreate 클래스는 테이블 형태로 폼을 구성하기 때문)

In [6]:
{% extends 'blog/base.html' %}

{%  block head_title %}Create Post - Blog{% endblock %}

{% block main_area %}
    <h1>Create New Post</h1>
    <hr/>
    <form method="post", enctype="multiplart/form-data">{% csrf_token %}
        <table>
            {{form}}
        </table>
        <button type="submit" class="btn btn-dark float-end">Submit</button>
    </form>
{% endblock %}  

SyntaxError: invalid syntax (<ipython-input-6-5a8f0c59018a>, line 1)

form 태그를 설펴보자 form 태그는 사용자가 기록한 값을 서버로 보내는 용도다. 서버로 값을 보내는 방식은 GET과 POST가 있는데 여기서는 POST방식을 사용한다.  
ectype="multiplart/form-data"는 파일도 같이 전송한다는 의미이다. (그냥 쓰자).  
> 원래 HTML 문법에서는 form 태그를 쓸 때 이 form 내용이 어느 URL로 전송되어야 하는지 나타내는 action이 포함되어 있어야 한다. 
예를들어 user가 /blog/create_post/를 통해 처리되기 원한다면 actions="/blog/create_post/"를 추가해줘야하지만 장고는 이런 부분을 처리해 주므로 쓰지않아도 된다.  

{% csrf_token %}은 웹사이트를 CSRF 공격으로부터 보호하기 위해 장고가 제공하는 기능이다. 장고에서 폼을 이용할 때는 이 부분을 꼭 form 태그안에 넣어야한다.  
>GET과 POST :  
방문자가 페이지에 입력한 값을 서버에 전달해서 DB에 저장하려고 할 때 POST가 아니라 GET 방식으로도 서버에 값을 전달하고 DB에 저장할 수 있다. 예를들어 blog/create_post/로 서버에 요청할 때 다음과 같이 입력하면 서버에서는 물음표(?) 뒤에 있는 내용을 처리해 
title=애국가, content="동해물과 백두산이" 라는 값을 DB에 저장한다(%20은 띄워쓰기이다.)
>> http://example/?title=애국가&content=동해물과%20백두산이%20마르고%20닳도록  
GET은 간편한 방식이지만 한계가 있다. 우선 길이에 제한이 있고 장문의 텍스트를 보낼 수 없다. 게다가 첨부파일을 서버에 저장 할 때 GET은 적합하지 않다. 이럴 때는 POST를 사용한다. POST는 URL 정보가 노출된 상태로 보내지 않고 HTTP의 body에 정보를 담아 전송한다.

테스트가 성공하면 웹브라우저로 들어가본다. 뒤에/blog/create_post/를 입력해 작성페이지가 들어가지면 성공이다.
#### (5) 템플릿 모양 개선하기
base.html을 이용하기 때문에 오른쪽에 검색창과 카테고리 카드도 위치하게 된다. 게다가 카테고리 카드는 빈 상태로 나타난다. 글을 쓰기에 집중하려면 넓은공간이 더 나을것이다. 왼쪽과 오른쪽을 나누지않고 12칸 전체를 활용하는 base_full_width.html을 새로만들자 지금까지 활용한 base.html을 복사하면 된다. 그리고 row태그 아래에  부분을 모두 삭제 하고 div class="col-12" id="main-area로 전체 영억을 다 활용하자

In [7]:
<div class="row">
    <div class="col-12" id="main-area">
    {% block main_area %}
    {% endblock %}
    </div>                     
</div>

SyntaxError: invalid syntax (<ipython-input-7-b26a090fd159>, line 1)

그리고 post_form.html에서 base.html을 base_full_width.html로 수정하자.
### 로그인한 방문자만 포스트를 작성할 수 있게 만들기
포스트 작성 페이지에 검색창과 카테고리 카드가 사라졌다.  이제 create_post 페이지에서 실제로 글을 작성해 보면 포스트의 작성자가 NONE으로 나오는 것을 알 수 있다.  만약 로그인한 사용자에 한해 글쓰기 권한을 부여하지 않는다면 익명의 사람이나, 광고 봇이 도배를 할 수도 있으므로 로그인한 경우에만 포스트 작성 페이지에 접근이 가능하게 수정하자.
#### (1) 테스트 코드 작성하기
로그인 하지않은 사용자에게 create_post 페이지가 열리지 않게 하기 위해 test_create_post()함수를 수정한다.  

In [8]:
def test_create_post(self):
    # 로그인을 하기 전 (status code가 200이면 안된다.)
    response=self.client.get('/blog/create_post/')
    self.assertNotEqual(response.status_code, 200)

    # 로그인 한디.
    self.client.login(username='trump', password='somepassword')

    response=self.client.get('/blog/create_post/')
    self.assertEqual(response.status_code, 200)
    soup=BeautifulSoup(response.content, 'html.parser')

    self.assertEqual('Create Post - Blog', soup.title.text)
    main_area=soup.find('div', id='main-area')
    self.assertIn('Create New Post',main_area.text)

로그인을 하지 않은 사람은 status가 200이면 안된다.(정상페이지 출력X)그리고 client의 login() 함수를 사용하면 (setUp()함수에서 user를 생성할 때 만든 id, password를 인자로 넣으면 된다.) 로그인이 된것이고 정상 페이지가 출력되게 해야 한다.
#### (2) views.py의 PostCreate에 LoginRequiredMixin 추가하기
로그인 상태에 따라 포스트 작성 페이지가 열리는지 확인하는 테스트 코드를 작성하였고 이제 views.py 에서 PostCreate 클래스에 장고에서 제공하는 LoginRequiredMixin를  import후, 매개변수로 추가하여  로그인 했을 때만 정상 페이지가 보이게 해주면 된다. 
>Mixin 클래스로 추가 상속받기 :  
이미 PostCreate 클래스는 장고가 제공하는 CreateView 클래스를 상속 받았는데, LoginRequiredMixin 를 상속 받은 것 처럼 LoginRequiredMixin의 메서드를 사용하고 있다. 이는 Mixin이라는 개념 때문인데 클래스에서 Mixin을 사용하면 다른 클래스의 메서드(함수)를 추가할 수 있다. CBV로 뷰를 구성하는 경우에 로그인 했을 때만 작동하는 LoginRequiredMixin 을 장고에서 이미 제공하고 있으니 단순히 PostCreate에 추가하는 것 만으로도 기능을 구현할 수 있다.

In [11]:
from django.contrib.auth.mixins import LoginRequiredMixin

class PostCreate(LoginRequiredMixin, CreateView):

ModuleNotFoundError: No module named 'django'

이제 test는 OK가 나오고 서버를 실행한후 웹에 들어간 뒤 로그아웃을 하고 접속을 하면 원하는대로 접근이 되지 않는다.   
URL은 보면 /accounts/login/?next=/blog/create_post/로 바뀌는데 이 URL로 들어가면 로그인 페이지가 나오게 하는것으로 나중에 구현할 예정이다.
#### (3) 테스트 코드 수정하기
이제 작성후 submit 버튼을 클릭하는 것까지 테스트하기 위해 구현해보자. test_create_post() 함수의 가장 아래에 다음의 코드를 추가해준다.

In [12]:
self.client.post(
    '/blog/create_post/',
    {
        'title' : 'Post Form 만들기',
        'content' : "Post Form 페이지를 만듭시다.",
    }
)
last_post=Post.objects.last()
self.assertEqual(last_post.title, "Post Form 만들기")
self.assertEqual(last_post.author.username, "trump")


NameError: name 'self' is not defined

#### 코드 해석
self.client.post() 함수는 첫번째 인수인 해당경로로 두번째 인수인 딕셔너리 정보를 POST 방식으로 보낸다. Post모델로 만든 폼은 title과 content 필드를 필수로 채워야 작동한다. created_at이나 author 같은 필드는 자동으로 채워지게 구현할 예정이다.  

Post.objects.last()는 Post 레코드 중 맨 마지막 레코드를 가져와 last_post 변수에 저장한다. POST방식으로 보낸 정보가 마지막으로 저장되었다면, 아래의 assertEqual 함수 2개가 정상적으로 통과해야한다. 아직은 autor필드를 자동으로 채우지않아 오류가 발생한다.
### 자동으로 author 필드 채우기
이제 자동으로 로그인한 user의 정보를 author필드에 채우기 위해 CreateView에서 제공하는 form_valid() 함수를 활용하는 방법에 대해 알아보자.
PostCreate 클래스는 CreateView를 상속받았으므로 제공한 폼에 사용자가 내용을 입력하면 form_valid()함수가 실행 된다.  
form_vlaid()함수는 방문자가 폼에 담아 보낸 유효한 정보를 사용해 포스트를 만들고, 이 포스트의 고유 경로로 보내주는(redirect)역할을 한다.
#### (1) form_valid() 함수 활용하기
다음과같이 form_valid()함수를 재 정의하여 기능을 확장할 수 있다. blog/views.py의 PostCreate에 다음의 코드를 주가하자.

In [13]:
class PostCreate(LoginRequiredMixin, CreateView):
    model=Post
    fields=['title', 'hook_text', 'content', 'head_image', 'file_upload', 'category']

    def form_valid(self, form):
        current_user=self.request.user
        if current_user.is_authenticated:
            form.instance.author=current_user
            return super(PostCreate, self).form_valid(form)
        else:
            return redirect('/blog/')

NameError: name 'LoginRequiredMixin' is not defined

#### 코드 해석
self.request.user는 웹 사이트의 방문자를 의미한다. 방문자라 로그인 상태인지 아닌지는 is_authenticated로 알 수 있다. 로그인을 한 상태라면 form에서 생성한  
instatnce(새로 생성한 포스트)의 author 필드에 current_user(현재 접속한 방문자)를 담는다. 그 상태에서 CreateView의 기본 form_valid()함수의 form을 인자로 보내 처리한다. 만약 방문자가 로그인을 하지 않았다면 redirect() 함수를 이용해 /blog/경로로 돌려보낸다.  

테스트 해보면 OK가 나온다. 다시 웹에 접속하고 submit버튼을 클릭해보면 새로 생성된 포스트의 상세 페이지로 바로 넘어간다. 생성된 포스트에는 작성자명이 추가된것을 확인할 수 있다.
### 스태프만 포스트 작성할 수 있게 만들기
댓글은 누구나 작성할 수 있어도, 모든 회원이 글을 쓰는 것이 아닌 최고 관리자 or 스태프만 작성할 수 있게 만들어 보자.
#### (1) 테스트 코드 작성
관리자 페이지에 USER 메뉴에 들어가면 사용자의 등급을 일반 사용자와 스태프, 최고 관리자 까지 부여할 수 잇다. 이 권한 여부에 따라 포스트 작성 페이지에 접근할 수 있는지 없는지를 코드로 작성하면 된다. 이 경우는 UserPassesTestMixin을 사용하면 된다. 적용하기에 앞서 테스트 코드를 수정하자. 먼저 biden에게 스태프 권한을 trump는 일반 사용자로 설정한다.

In [14]:
# biden은 스태프 권한 부여
self.user_biden.is_staff=True
self.user_biden.save()

def test_create_post(self):
    # 로그인을 하기 전 (status code가 200이면 안된다.)
    response=self.client.get('/blog/create_post/')
    self.assertNotEqual(response.status_code, 200)

    # staff 권한이 없는 trump가 로그인 한디.
    self.client.login(username='trump', password='somepassword')        
    response=self.client.get('/blog/create_post/')
    self.assertEqual(response.status_code, 200)

    # staff인 biden으로 로그인한다.
    self.client.login(username='biden', password='somepassword')
    soup=BeautifulSoup(response.content, 'html.parser')
    self.assertEqual('Create Post - Blog', soup.title.text)
    main_area=soup.find('div', id='main-area')
    self.assertIn('Create New Post',main_area.text)

    self.client.post(
        '/blog/create_post/',
        {
            'title' : 'Post Form 만들기',
            'content' : "Post Form 페이지를 만듭시다.",
        }
    )
    last_post=Post.objects.last()
    self.assertEqual(last_post.title, "Post Form 만들기")
    self.assertEqual(last_post.author.username, "biden")


NameError: name 'self' is not defined

#### (2) views.py에 UserPassesTestMixin 추가하기.
blog/views.py에서 PostCreate클래스를 수정한다. UserPassestestMixin을 인자로 추가하고. test_func()함수를 추가해 이 페이지에 접근 가능한 사용자를 superuser(최고 관리자) 또는 staff(스태프)로 제한한다. form_vlaid()에서도 로그인한 사용자가 superuser이거나 staff인 경우에만 동작하도록 수정한다.

In [15]:
class PostCreate(LoginRequiredMixin, UserPassesTestMixin,CreateView):
    model=Post
    fields=['title', 'hook_text', 'content', 'head_image', 'file_upload', 'category']

    def test_func(self):
        return self.request.user.is_superuser or self.request.user.is_staff

    def form_valid(self, form):
        current_user=self.request.user
        if current_user.is_authenticated:
            form.instance.author=current_user
            return super(PostCreate, self).form_valid(form)
        else:
            return redirect('/blog/')

NameError: name 'LoginRequiredMixin' is not defined

이제 테스트를 하면 OK가 나온다.
### 새 포스트 작성 버튼 만들기
#### (1) 로그인 했을 때만 New Post 버튼 나타내기
post_list.html을 수정하자. 로그인 되어있는 경우 사용자가 최고관리자이거나 스태프 일때만 버튼이 보이도록 한다.

In [16]:
{% if user.is_authenticated %}
    {% if user.is_superuser or user.is_staff %}
        <a class="btn btn-dark float-end" href="/blog/create_post/" role="button"><i class="fas fa-pen">New Post</i></a>
    {% endif %}
{% endif %}

SyntaxError: invalid syntax (<ipython-input-16-4e4430be7c64>, line 1)

#### (2) 관리자 권한 계정으로 확인하기
버튼이 만들어진걸 확인할 수 있다. 시크릿 페이지로 접속하면 New Post버튼이 보이지 않는다.
#### (3) 스타프 권한 계정으로 확인하기
staff 권한을 가진 계정으로 접속해봐도 버튼이 잘보인다.
## 포스트 수정 페이지 만들기
이제 포스트를 수정하는 페이지도 추가해보자.
### 테스트 코드로 기본 요건 정의하기
### (1) 테스트 코드 작성하기
test_update_post() 함수로 작성해보자.

In [18]:
def test_update_post(self):
    # 1번과정.
    update_post_url=f'/blog/update_post.{self.post_003.pk}/'

    # 2번과정.
    # 로그인을 하기 전 (status code가 200이면 안된다.)
    response=self.client.get('/blog/create_post/')
    self.assertNotEqual(response.status_code, 200)

    # 로그인 했지만 작성자가 아닌 경우
    self.assertNotEqual(self.post_003.author, self.user_trump)
    self.client.login(
        username=self.user_trump.username,
        password='somepassword'
    )        
    response=self.client.get(update_post_url)
    self.assertEqual(response.status_code, 403)

    # 작성자가(biden)이 접근하는 경우
    self.client.login(
        username=self.post_003.author.username,
        password='somepassword'
    )
    response=self.client.get(update_post_url)
    self.assertEqual(response.status_code, 200)
    soup=BeautifulSoup(response.content, 'html.parser')

    # 3번과정.
    self.assertEqual('Edit Post - Blog', soup.title.text)
    main_area=soup.find('div', id='main-area')
    self.assertIn('Edit Post',main_area.text)

    response=self.client.post(
        update_post_url,
        {
            'title' : '세 번째 포스트를 수정했습니다.',
            'content' : "안녕 세계? 우리는 하나!",
            'category' : self.category_music.pk
        },
        follow=True
    )
    soup=BeautifulSoup(response.content, 'html.parser')
    main_area=soup.find('div', id='main-area')
    self.assertEqual('세 번째 포스트를 수정했습니다.', main_area.text)
    self.assertEqual("안녕 세계? 우리는 하나!", main_area.text)
    self.assertEqual(self.category_music.name, main_area.text)

#### 코드 해석
1. 수정할 포스트는 setUp()함수에서 만든 self.post_003이다. URL의 형태는 /blog/update_post/포스트의 pk/로 정한다.
2. 이 페이지는 기존 포스트의 작성자만 접근 가능해야 한다. 로그인 하지 않는 경우는 200이 나와야 한다. 방문자가 로그인 했지만 작성자가 없으면 권한이 없음을 알리는 403 오류가 있어야 한다. 또한 post_003의 작성자인 biden이 접근할 때 페이지가 열리는지 테스트해야한다.
3. 제대로 페이지가 열리면 title, content, category가 수정한 내용과 일치하는지 검사한다.

#### (2) ulrs.py와 views.py 수정
urls.py에 추가할 내용은 다음과 같다. URL, 즉 /blog/update_post/포스트의 pk/로 접근할 때 views.py의 PostUdate 클래스를 사용하도록 해주어야 하고,  
마찬가지로 views.py에서도 PostUpdate 클래스를 만들어야 한다. 

In [19]:
# urls.py의 urlpatterns에 추가
path('update_post/<int:pk>/',views.PostUpdate.as_view()),

NameError: name 'path' is not defined

views.py에서는 PostUpdate 클래스를 CBV스타일로 구현한다. PostCreate와 동일하지만 CreateView대신 UpdateView를 사용한다. PostCreate는 새로운 Post 레코드의 author 필드를 로그인한 사용자로 채워주는 기능을 추가하기 위해 form_valid()를 사용했으나 PostUpdate에서는 이미 작성자가 존재하므로 사용하지 않는다.

In [20]:
class PostUpdate(LoginRequiredMixin, UpdateView):
    model=Post
    fields=['title', 'hook_text', 'content', 'head_image', 'file_upload', 'category']

NameError: name 'LoginRequiredMixin' is not defined

하지만 아직 포스트 작성자만 수정하는 기능을 구현하지 않아 test는 오류가 뜬다.
### 포스트 작성자만 수정할 수 있게 구현하기
포스트를 수정하는 페이지는 작성자 본인만 접근할 수 있게 한다. 방문자가 포스트의 작성자와 동일한지 확인하고 권한이 없다면 오류메시지를 나타낸다. 이렇게 구현하기 위해 CBV에서 제공하는 dispatch()메서드를 활용한다.
#### (1) dispatch()로 요청 방식 판단하기
dispatch() 메서드는 방문자가 웹 사이트 서버에 GET방식인지 POST 방식인지 판단한다. UpdateView의 경우 방문자가 서버에 GET 방식으로 들어오면 포스트를 작성할 수 있는  폼 페이지를 보내주고, 폼 내용을 담아서 POST 방식으로 들어오면 폼의 양식이 유효한지 확인 후, DB에 저장한다. 만약 권한이 없는 사용자가 PostUpdate를 사용하려한다면 GET방식이든 POST방식이든 접근할 수 없게 해야한다. 따라서 dispatch()가 실행 되는 순간 작성자가 맞는지 확인하도록 수정한다.

In [21]:
def dispatch(self, request, *args, **kwargs):
    if request.user.is_authenticated and request.user == self.get_object().author:
        return super(PostUpdate, self).dispatch(request, *args, **kwargs)
    else:
        raise PermissionDenied

#### 코드 해석
먼저 request.useris_authenticated 즉, 방문자는  로그인한 상태여야 한다.  
뒤의 self.get_object().author에서 get_object()는 UpdateView의 메서드중 하나로 pk를 가져온다. 이렇게 가져온 Post 인스턴스(레코드)의 author필드가 사용자와 동일하다면 dispatch() 메서드는 원래의 역할(GET or POST를 판단하고 DB에저장하는 역할)을 수행한다. 사용자가 로그인 하지않았나 작성자가 아니라면  권한이없음을 의미하도록
장고에서 제공하는 raise PermissionDenied를 실행한다. 이렇게 해두면 권한이 없는 경우 403오류 메시지가 나타난다.
#### (2) 템플릿 지정하기
test를 해보면 title이 'Create Post - blog' 라고 뜬다. 즉, 수정이 반영되지 않았다. PostUpdate에게 상속한 UpdateView는 '모델 명_form.html' 구조의 템플릿 파일을 사용하도록 기본 설정 되어있다. 그래서 PostUpdate에서도 PostCreate을 작성할때 만들어둔 post_form.html을 자동으로 찾아서 사용한 것이다.  

CBV로 뷰를 만들 때 template_name을 지정해 원하는 html파일을 템플릿 파일로 설정할 수 있었다. template_name을 다음과 같이 지정해준다.  

In [22]:
# class PostUpdate안에 추가
template_name='blog/post_update_form.html'

그리고 post_form.html을 복사해서 post_update_form.html로 이름을 바꾼뒤 Create를 Edit으로 바꿔준다. 이제 테스트가 OK로 나온다.
### 포스트 상세 페이지에 수정 버튼 추가하기
#### Edit Post 버튼 만들기
New Post 버튼과정과 거의 같다. 다만 로그인한 상태이며 작성자가 같다는 if문을 넣고, post_detail.html에만 추가해줘야한다.
## 태그 선택란 추가하기
PostCreate 클래스 fields 리스트에 'tags를 추가하고 포스트 작성 페이지에 들어가자 보면 현재 존재하는 태그만 선택할 수 있다. Post 레코드의 tags 필드가 ManyToManyField로 지정되어 있어 기본적으로 제공하는 폼이 이런 형태를 갖게 된것이다. 작성자가 태그를 직접 텍스트로 입력할 수 있게 만들어 보자.
### CreateView가 만들어준 내용 분석하기
PostCreate 클래스는 CreateView의 기능을 상속받아 사용했다. CreateView가 만들어준 내용을 알아야 직접 태그 선택란은 만들 수 있다.  
post_form.html에서 " {{form}} " 부분을 살펴보자. 

In [24]:
<tr><th><label for="id_title">Title:</label></th><td><input type="text" name="title" maxlength="30" required id="id_title"></td></tr>
<tr><th><label for="id_hook_text">Hook text:</label></th><td><input type="text" name="hook_text" maxlength="100" id="id_hook_text"></td></tr>
<tr><th><label for="id_content">Content:</label></th><td><textarea name="content" cols="40" rows="10" required id="id_content">
</textarea></td></tr>
<tr><th><label for="id_head_image">Head image:</label></th><td><input type="file" name="head_image" accept="image/*" id="id_head_image"></td></tr>
<tr><th><label for="id_file_upload">File upload:</label></th><td><input type="file" name="file_upload" id="id_file_upload"></td></tr>
<tr><th><label for="id_category">Category:</label></th><td><select name="category" id="id_category">

SyntaxError: invalid syntax (<ipython-input-24-3750a145657c>, line 1)

{{form}}이 있던 위치에 위와 같이 fields 리스트에 지정한 필드들이 테이블의 한 줄씩 차지하고 있다. 그리고 필드마다 tr태그로 테이블에서 한 줄을 부여하고, th로 필드명을 표시 후 td안에 input 요소를 필드의 데이터 타입에 맞게 만들어 제공하고 있다.
### 템플릿 파일에 input 추가하기
다음과 같이 post_form.html의 {{form}} 밑에 tags 필드를 추가한다. tr 태그로 한줄을 더 추가하고 그 안에 label과 input 요소를 넣어준다. input 요소에는 문자를 입력 받을 수 있게 type='text name="tags_str" id="id_tags_str"로 속성을 추가한다.

In [25]:
{{form}}
<tr>
    <th><label for="id_tags_str">Tags : </label></th>
    <td><input type="text" name="tags_str" id="id_tags_str"></td>
</tr>

SyntaxError: invalid syntax (<ipython-input-25-c3e8c4503a9b>, line 2)

tag를 입력란이 생긴것을 볼 수 있다. views의 fields에서 tags를 지워주자
### 포스트 작성 페이지에 태그 입력란 추가하기
이제 태그를 텍스트로 작성해 추가할 수 있는 기능을 구현해보자. 예를들어 입력란에 '파이선; 세미콜론,쉼표'를 작성하고 submit 버튼을 누르면 '파이썬','세미콜론','쉼표' 라는 태그가 추가된다. 이 중에서 '파이썬'태그가 이미 존재하므로 새로 생성하지 않고, '세미콜론'과 '쉼표'만 새로 생상한 다음 포스트에 연결되도록 한다.
#### (1) 테스트 코드 작성하기
test 코드는 test_create_post()함수에 내용을 추가한다.

In [26]:
    def test_create_post(self):
        # 로그인을 하기 전 (status code가 200이면 안된다.)
        response=self.client.get('/blog/create_post/')
        self.assertNotEqual(response.status_code, 200)
        
        # staff 권한이 없는 trump가 로그인 한디.
        self.client.login(username='trump', password='somepassword')        
        response=self.client.get('/blog/create_post/')
        self.assertNotEqual(response.status_code, 200)

        # staff인 biden으로 로그인한다.
        self.client.login(username='biden', password='somepassword')
        response=self.client.get('/blog/create_post/')
        self.assertEqual(response.status_code, 200)
        soup=BeautifulSoup(response.content, 'html.parser')

        self.assertEqual('Create Post - Blog', soup.title.text)
        main_area=soup.find('div', id='main-area')
        self.assertIn('Create New Post',main_area.text)

        # input존재 확인
        tag_str_input=main_area.find('input', id='id_tags_str')
        self.assertTrue(tag_str_input)

        # post 생성 (제목 내용 태그)
        self.client.post(
            '/blog/create_post/',
            {
                'title' : 'Post Form 만들기',
                'content' : "Post Form 페이지를 만듭시다.",
                'tags_str' : 'new tag; 한글 태그, python'
            }
        )
        # post검사
        self.assertEqual(Post.objects.count(), 4)
        last_post=Post.objects.last()
        self.assertEqual(last_post.title, "Post Form 만들기")
        self.assertEqual(last_post.author.username, "biden")

        # tag검사
        self.assertEqual(last_post.tags.count(), 3)
        self.assertTrue(Tag.objects.get(name='new tag'))
        self.assertTrue(Tag.objects.get(name='한글 태그'))
        self.assertEqual(Tag.objects.count(), 5)

#### 코드 해석
1. main-area에 id='id_tags_str'인 input이 존재하는지 확인한다. 
2. form을 채워 서버에 POST 방식으로 전송/요청하기 전에 태그가 몇개 존재하는지 확인한다.(3개여야한다.)
3. self.client.post에 tags_str을 추가하고 'new tag; 한글태그, python'을 담아 보내기로 했다 (세미콜론 쉼표 둘다 구분자로 사용가능  
4. setUp에서 만든 3개의 태그, test_post_create에서 사용한 3개의 태그 중 새로운 태그는2개 즉, 5개의 태그가 있어야한다.

#### (2) views.py 수정
이제 tag_str을 처리해야 한다. post_form.html에서 추가한 name='tags_str'인 input 요소에 입력된 값을 가져오기 위해 form_valid()기능을 추가한다.

In [27]:
from django.utils.text import slugify # 5번

def form_valid(self, form):
    current_user=self.request.user
    if current_user.is_authenticated:
        form.instance.author=current_user
        response=super(PostCreate, self).form_valid(form)  # 1번

        tags_str=self.request.POST.get('tags_str') # 2번
        if tags_str: # 3번
            tags_str=tags_str.strip()

            tags_str=tags_str.replace(',',';')
            tags_list=tags_str.split(';')

            for t in tags_list:
                t=t.strip() # 4번
                tag, is_tag_created = Tag.objects.get_or_create(name=t)# 5번
                if is_tag_created:# 6번
                    tag.slug=slugify(t,allow_unicode=True)
                    tag.save()
                self.object.tags.add(tag)# 7번

        return response # 8번
    else:
        return redirect('/blog/')

ModuleNotFoundError: No module named 'django'

#### 코드해석
1. 태그 관련 작업을 하기전 CreateView의 form_valid()함수의 결괏은 response 변수에 임시로 담아둔다.
2. 장고가 자동으로 작성한 post_form.html의 form 은 method='post'로 설정되어있다. 이 폼안에 name='tags_str'인 input을 추가 했으니 방문자가 submit 버튼을 클릭했을 때 이 값 역시 POST 방식으로 PostCreate까지 전달되어 있는 상태이다. 이 값은 self.request.POST.get('tags_str')로 받을 수 있다. POST 방식으로 전달된 정보 중  name='tags_str'인 input 값을 가져오라는 뜻이다.
3. 이 값이 빈칸이면 태그와 관련된 동작은 하지 않는다. 따라서 if문으로 조건을 걸어 둔다. tags_str이 존재한다면 쉼표와(,) 세미콜론(;)으로 모두 변경후 split하여 list에 담는다.
4. tags_list에 담겨 있는 요소들은 문자열 형태이므로 Tag모델의 인스턴스로 변환하기 위해 strip()으로 앞뒤 공백을 제거한다.
5. 이 값을 name 태그로 갖는 태그가 있다면(이미 존재하는 태그라면) 가져오고 아니면 새로만든다. get_or_create() 는 두가지 값을 동시에 return한다. 첫 번째는 Tag모델의 인스턴스이고, 두 번째는 이 인스턴스가 새로 생성 되었는지를 나타내는 bool형태의 값이다.
6. 만약 name을 갖는 태그가 없어 새로생성했다면 아직 slug값이 없으므로 slug값도 생성해야 한다. 관리자 페이지에서 작업할때는 자동으로 생성해줬지만 이번엔 get_or_create() 메서드를 사용했기 때문에 slug 값을 따로 만들어줘야한다. 이럴때 slugify()함수를 사용한다. 한글 태그도 slug를 만들 수 있게 allow_unicode=True로 설정한다. 이 값을 태그의 slug에 부여하고 저장하면 name, slug 필드 모두 채운 상태로 저장된다. slugify()를 쓰려면 import도 해줘야한다.
7. 기존의 태그를 쓰든 새로운 태그를 만들든 새로 만드는 포스트의 tags 필드에 추가를 해줘야한다. 이때 self.object가 새로 만든 포스트를 의미한다.
8. 원하는 작업이 끝나면 새로 만든 포스트의 페이지로 이동 해야하므로 response에 담아뒀든 CreateView의 form_valid() 결과를 return한다.  

> 태그 관련 내용에 앞서 form_valid() 함수를 사용하는 이유는?  
CreateView, UpdateView의 form_valid()함수는 폼 안에 들어온 값을 바탕으로 모델에 해당하는 인스턴스를 만들어 DB에 젖아한 다음 그 인스턴스의 경로로 리다이렉트를 하는 역할을 한다. 이 때 CreateView의 form_valid()함수를 오버라이딩 해서 사용한 이유는 DB에 저장하기전 폼에 담고있지 않았던 author정보를 추가하기 위해서다. 이번에는 태그 까지 추가하려 한것이다.  
문제는 포스트에 태그를 추가하려면 포스트가 이미 DB에 저장된 pk를 부여받은 다음 이어야 한다. Post모델과 Tag모델은 다대다 관계 이므로 Post레코드가 이미 존재해야 한다. 그래서 태그와 관련된 작업을 하기전에 CreateView의 form_valid()함수를 사용하고 그 결과를 response 변수에 임시로 담아 둔것이다. 새로 저장된 포스트는 self.object라고 장고가 구성하므로 여기에서 tags필드에 원하는 태그를 추가할 수 있다.  

다시 테스트해서 OK가 뜨면된다. 그 후 직접 서버를 실행하고 웹에 들어가보면 Tags를 쓸 수 있는 칸이 생겼다.
### 포스트 수정페이지에 태그 입력란 추가하기
#### (1) 테스트 코드 작성

In [28]:
# 테그 수정
tag_str_input=main_area.find('input', id='id_tags_str')
self.assertTrue(tag_str_input)
self.assertIn('파이썬 공부; python',tag_str_input.attrs['value'])

response=self.client.post(
    update_post_url,
    {
        'title' : '세 번째 포스트를 수정했습니다.',
        'content' : "안녕 세계? 우리는 하나!",
        'category' : self.category_music.pk,
        'tags_str' : '파이썬 공부; 한글 태그, some tag'
    },
    follow=True
)
soup=BeautifulSoup(response.content, 'html.parser')
main_area=soup.find('div', id='main-area')
self.assertIn('세 번째 포스트를 수정했습니다.', main_area.text)
self.assertIn("안녕 세계? 우리는 하나!", main_area.text)
self.assertIn(self.category_music.name, main_area.text)

# 태그 테스트
self.assertIn('파이썬 공부',main_area.text)
self.assertIn('한글 태그', main_area.text)
self.assertIn('some tag', main_area.text)
self.assertNotIn('python', main_area.text)

NameError: name 'main_area' is not defined

test_post_create 처럼 id_tags_str의 id를 가진 input이 있는지 확인한다. 기존의 태그가 저장되어 있어야하므로 확인한 input에 self.post_003의 태그가 들어있는지 확인한다. self.client.post에서는 form에 tags_str을 추가하고 파이썬 공부, new tag, 한글 태그를 추가한다. 정상적으로 PostUpdate 클래스가 작동되면 이 내용은 self.post_003의 포스트 상세 페이지로 리다이렉트 되고 기존의 python 태그가 사라지고 한글 태그와 some 태그가 추가되어야 한다.  
물론 아직은 테스트하면 input을 찾을 수 없다고 뜬다.
#### (2) 템플릿 수정하기
post_update_form.html을 수정한다.

In [29]:
{{form}}
<tr>
    <th><label for="id_tags_str">Tags : </label></th>
    <td><input type="text" name="tags_str" id="id_tags_str"></td>
</tr>

SyntaxError: invalid syntax (<ipython-input-29-c3e8c4503a9b>, line 2)

#### (3) 뷰 수정하기
tag_str_input의 value에 현재 포스트의 태그가 나열되어 있어야 사용자가 수정하지않고 submit 버튼을 눌렀을 때 현재 상태가 그대로 유지될 수 있다. 수정 페이지에 들어갔을 때 Tags에 파이썬, 한글 태그, new tag가 미리 입력되어 있야한다. post_update_form.html에서 input요소에 value라는 속성을 추가하면된다. 

In [30]:
value="{{tags_str_defalut}}"

tags_str_defalut를 만들기위해 view.py로 들어간다. CBV로 뷰를 만들 때 템플릿으로 추가 인자를 넘기려면 get_context_data()를 이용한다고 앞에서 배웠다. 만약 이 Post레코드(self.object)로 에 tags가 존재한다면 이 tags의 name을 리스트 형태로 담은 후 리스트의 값들을 세미콜론(;)으로 결합하여 하나의 문자열로 만들고 context['tags_str_default']에 담아 return하면 템플릿의 해당위치가 채워진다.

In [32]:
def get_context_data(self, **kwargs):
        context=super(PostUpdate,self).get_context_data()
        if self.object.tags.exists():
            tag_str_list=list()
            for t in self.object.tags.all():
                tag_str_list.append(t.name)
                context['tags_str_default']=';'.join(tag_str_list)
        return context

test 해보면 한글 태그를 찾을 수없다고 한다. PostUpdate에서 form_valid를 추가하지 않았기 때문이다. PostCreate에서 작성한 삼수를 조금만 수정하면된다.
먼저 self.object로 갖겨온 태그를 모두 삭제한다는 내용을 추가한다. 또한 이미 존재하는 tags를 제거하는 기능은 구현되어 있지 않기 때문에 애초에 포스트의 태그를 모드 비우고 tags_str로 들어온 값으로 다시 채우면 간단하게 문제를 해결할 수 있다. 그리고 author역시 가져올 필요가없다.

In [34]:
def form_valid(self, form):
    response=super(PostUpdate, self).form_valid(form)
    self.object.tags.clear() 

    tags_str=self.request.POST.get('tags_str')
    if tags_str:
        tags_str=tags_str.strip()

        tags_str=tags_str.replace(',',';')
        tags_list=tags_str.split(';')

        for t in tags_list:
            t=t.strip()
            tag, is_tag_created = Tag.objects.get_or_create(name=t)
            if is_tag_created:
                tag.slug=slugify(t,allow_unicode=True)
                tag.save()
            self.object.tags.add(tag)

    return response

이제 테스트를 하면 OK가 나온다.